In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import time
from tqdm import tqdm
import numpy as np

pd.set_option('display.max_columns', 500) 

In [2]:
test = pd.read_csv('./test.csv')

In [3]:
test.car_url[0]

'https://auto.ru/cars/used/sale/skoda/octavia/1100575026-c780dc09/'

## Парсинг ссылок (Москва)

In [4]:
brand_list = list(test.brand.unique())
brand_list

['SKODA',
 'AUDI',
 'HONDA',
 'VOLVO',
 'BMW',
 'NISSAN',
 'INFINITI',
 'MERCEDES',
 'TOYOTA',
 'LEXUS',
 'VOLKSWAGEN',
 'MITSUBISHI']

In [ ]:
url = 'https://auto.ru/moskva/cars/'

In [2]:
links_msk = []
for brand in brand_list:
    print(brand)
    for i in tqdm(range(100)):
        new_url = url + brand + '/used/?page=' + str(i+1)
        res = requests.get(new_url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(res.text, 'html.parser')
        blocks = soup.find_all('div', class_='ListingItem-module__container')
        h3 = list(map(lambda x: x.find('h3', 'ListingItemTitle-module__container'), blocks))
        links_msk += list(map(lambda x: x.find('a', 'ListingItemTitle-module__link')['href'], h3))

In [8]:
links_msk = set(links_msk)

## Парсинг ссылок (СПб)

In [56]:
brand_list = list(test.brand.unique())
brand_list

['SKODA',
 'AUDI',
 'HONDA',
 'VOLVO',
 'BMW',
 'NISSAN',
 'INFINITI',
 'MERCEDES',
 'TOYOTA',
 'LEXUS',
 'VOLKSWAGEN',
 'MITSUBISHI']

In [50]:
url = 'https://auto.ru/sankt-peterburg/cars/'

In [1]:
links_spb = []
for brand in tqdm(brand_list):
    for i in range(100):
        new_url = url + brand + '/used/?page=' + str(i+1)
        res = requests.get(new_url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(res.text, 'html.parser')
        time.sleep(0.3)
        blocks = soup.find_all('div', class_='ListingItem-module__container')
        h3 = list(map(lambda x: x.find('h3', 'ListingItemTitle-module__container'), blocks))
        links_spb += list(map(lambda x: x.find('a', 'ListingItemTitle-module__link')['href'], h3))

In [53]:
links_spb = set(links_spb)

## Парсинг объявлений

In [5]:
links = links_spb + links_msk

In [29]:
df_train = pd.DataFrame() 

In [3]:
for url in tqdm(links):
    try:
        res = requests.get(url)
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, 'html.parser')
        
        script1 = soup.find('script', type='application/ld+json').contents[0]
        script1_json = json.loads(script1)
        script2 = soup.find('script', id='initial-state').contents[0]
        script2_json = json.loads(script2)
        ul = soup.find('ul', class_='CardInfo')
    except:
        continue
        
    train = {}

    try:
        train['bodyType'] = script1_json['bodyType']   
    except:
        np.nan        
    try:
        train['brand'] = script1_json['brand']
    except:
        np.nan   
    try:
        train['car_url'] = script1_json['offers']['url']
    except:
        np.nan   
    try:
        train['color'] = script1_json['color']
    except:
        np.nan   
    try:
        train['complectation_dict'] = np.nan
    except:
        np.nan    
    try:
        train['description'] = script1_json['description']
    except:
        np.nan    
    try:
        train['engineDisplacement'] = script1_json['vehicleEngine']['engineDisplacement']
    except:
        np.nan   
    try:
        train['enginePower'] = script1_json['vehicleEngine']['enginePower']
    except:
        np.nan   
    try:
        train['equipment_dict'] = str(script2_json['card']['vehicle_info']['equipment'])
    except:
        np.nan    
    try:
        train['fuelType'] = script1_json['vehicleEngine']['fuelType']
    except:
        np.nan   
    try:
        train['image'] = script1_json['image']
    except:
        np.nan    
    try:
        train['mileage'] = script2_json['card']['state']['mileage']
    except:
        np.nan   
    try:
        train['modelDate'] = script1_json['modelDate']
    except:
        np.nan    
    try:
        train['model_info'] = str(script2_json['card']['vehicle_info']['model_info'])
    except:
        np.nan   
    try:
        train['model_name'] = script2_json['card']['vehicle_info']['model_info']['code']
    except:
        np.nan
        
    try:
        train['name'] = script2_json['card']['vehicle_info']['tech_param']['human_name']
    except:
        np.nan    
    try:
        train['numberOfDoors'] = script1_json['numberOfDoors']
    except:
        np.nan    
    try:
        train['parsing_unixtime'] = int(time.time())
    except:
        np.nan    
    try:
        train['priceCurrency'] = script1_json['offers']['priceCurrency']
    except:
        np.nan    
    try:
        train['productionDate'] = script1_json['productionDate']
    except:
        np.nan    
    try:
        train['sell_id'] = script2_json['card']['id']
    except:
        np.nan    
    try:
        train['super_gen'] = str(script2_json['card']['vehicle_info']['tech_param'])
    except:
        np.nan    
    try:
        train['vehicleConfiguration'] = script1_json['vehicleConfiguration']
    except:
        np.nan    
    try:
        train['vehicleTransmission'] = script1_json['vehicleTransmission']
    except:
        np.nan    
    try:
        train['vendor'] = np.nan
    except:
        np.nan    
    try:
        train['Владельцы'] = ul.find_all('li', class_='CardInfoRow_ownersCount')[0].find_all('span')[1].text
    except:
        np.nan    
    try:
        train['Владение'] = ul.find_all('li', class_='CardInfoRow_owningTime')[0].find_all('span')[1].text
    except:
        np.nan    
    try:
        train['ПТС'] = ul.find_all('li', class_='CardInfoRow_pts')[0].find_all('span')[1].text
    except:
        np.nan    
    try:
        train['Привод'] = ul.find_all('li', class_='CardInfoRow_drive')[0].find_all('span')[1].text
    except:
        np.nan    
    try:
        train['Руль'] = ul.find_all('li', class_='CardInfoRow_wheel')[0].find_all('span')[1].text
    except:
        np.nan    
    try:
        train['Состояние'] = ul.find_all('li', class_='CardInfoRow_state')[0].find_all('span')[1].text
    except:
        np.nan
    try:
        train['Таможня'] = ul.find_all('li', class_='CardInfoRow_customs')[0].find_all('span')[1].text
    except:
        np.nan
    try:
        train['price'] = script1_json['offers']['price']
    except:
        np.nan 
        
    df_train = df_train.append([train])

In [32]:
df_train.to_csv('train.csv', index=False)